In [12]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

time = "7pm_"
# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

In [13]:
## FanDuel Player Pool



contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

# fd = glob.glob('*-players-list.csv')[0]

fd_upload = glob.glob('*upload-template.csv')[0]
df = pd.read_csv(fd_upload, header=6, usecols=[13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,30])
df.to_csv("FDSalaries_"+ contest + str(today)+".csv", index=False)
# os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [14]:
## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

fanduel_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel"

fd_upload = glob.glob('*upload-template.csv')[0]
os.rename(fd_upload, "FD_Upload_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FD_Upload_"+ contest + str(today)+".csv",
           fanduel_upload + "/"+ "FD_Upload_"+ contest + str(today)+".csv")

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\Upload\\FanDuel/FD_Upload_PlayerPool_03.18.2022.csv'

In [15]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [16]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id', 'Salary']], 
              left_on='Player', 
              right_on='Nickname')

In [17]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Salary
0,Jordan Clarkson,UTA,LAC,34.5,PG/SG,"5,000",34.3,9.3,43.2,65.8,MAIN,Jordan Clarkson,PG/SG,73111-20640,5000
1,Fred VanVleet,TOR,LAL,37.0,PG,"7,200",42.2,8.4,51.8,30.2,MAIN,Fred VanVleet,PG,73111-24247,7200
2,Nikola Jokic,DEN,CLE,34.0,C,"11,200",57.1,7.3,66.7,9.4,MAIN,Nikola Jokic,C,73111-55062,11200
3,CJ McCollum,NOP,SAS,36.0,SG,"8,700",47.1,7.3,54.9,14.6,MAIN,CJ McCollum,SG,73111-19067,8700
4,Julius Randle,NYK,WAS,36.0,PF/C,"8,600",45.5,6.1,54.5,24.2,MAIN,Julius Randle,PF/C,73111-42669,8600


In [18]:
# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
# df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Name + ID'] = df['Id'] + ":" + df['Player']

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Salary,G Position,F Position,list,Name + ID
0,Jordan Clarkson,UTA,LAC,34.5,PG/SG,"5,000",34.3,9.3,43.2,65.8,MAIN,Jordan Clarkson,PG/SG,73111-20640,5000,G,NaN,PG/SG/G,73111-20640:Jordan Clarkson
1,Fred VanVleet,TOR,LAL,37.0,PG,"7,200",42.2,8.4,51.8,30.2,MAIN,Fred VanVleet,PG,73111-24247,7200,G,NaN,PG/G,73111-24247:Fred VanVleet
2,Nikola Jokic,DEN,CLE,34.0,C,"11,200",57.1,7.3,66.7,9.4,MAIN,Nikola Jokic,C,73111-55062,11200,NaN,NaN,C,73111-55062:Nikola Jokic
3,CJ McCollum,NOP,SAS,36.0,SG,"8,700",47.1,7.3,54.9,14.6,MAIN,CJ McCollum,SG,73111-19067,8700,G,NaN,SG/G,73111-19067:CJ McCollum
4,Julius Randle,NYK,WAS,36.0,PF/C,"8,600",45.5,6.1,54.5,24.2,MAIN,Julius Randle,PF/C,73111-42669,8600,NaN,F,PF/C/F,73111-42669:Julius Randle


In [19]:
df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)

df = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_'+ str(today) + ".csv")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Salary,G Position,F Position,list,Name + ID
0,Jordan Clarkson,UTA,LAC,34.5,PG/SG,"5,000",34.3,9.3,43.2,65.8,MAIN,Jordan Clarkson,PG/SG,73111-20640,5000,G,NaN,PG/SG/G,73111-20640:Jordan Clarkson
1,Fred VanVleet,TOR,LAL,37.0,PG,"7,200",42.2,8.4,51.8,30.2,MAIN,Fred VanVleet,PG,73111-24247,7200,G,NaN,PG/G,73111-24247:Fred VanVleet
2,Nikola Jokic,DEN,CLE,34.0,C,"11,200",57.1,7.3,66.7,9.4,MAIN,Nikola Jokic,C,73111-55062,11200,NaN,NaN,C,73111-55062:Nikola Jokic
3,CJ McCollum,NOP,SAS,36.0,SG,"8,700",47.1,7.3,54.9,14.6,MAIN,CJ McCollum,SG,73111-19067,8700,G,NaN,SG/G,73111-19067:CJ McCollum
4,Julius Randle,NYK,WAS,36.0,PF/C,"8,600",45.5,6.1,54.5,24.2,MAIN,Julius Randle,PF/C,73111-42669,8600,NaN,F,PF/C/F,73111-42669:Julius Randle


# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG = 2
- TotalPosition_SG = 2
- TotalPosition_SF = 2
- TotalPosition_PF =2 
- TotalPosition_C =1


In [20]:
## Unnests FD Positions into each Row (explode in Python) ##


## Update Salary to INT 
# df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['FD Position'] = df['FD Position'].astype(str)
df['FD Position'] = df['FD Position'].str.split('/')

df = df.explode('FD Position')

df['Position + Id'] = df['FD Position'] + "_" + df['Id']

df.head()

df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [21]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Salary,G Position,F Position,list,Name + ID,Position + Id
0,Jordan Clarkson,UTA,LAC,34.5,PG,"5,000",34.3,9.3,43.2,65.8,MAIN,Jordan Clarkson,PG/SG,73111-20640,5000,G,NaN,PG/SG/G,73111-20640:Jordan Clarkson,PG_73111-20640
0,Jordan Clarkson,UTA,LAC,34.5,SG,"5,000",34.3,9.3,43.2,65.8,MAIN,Jordan Clarkson,PG/SG,73111-20640,5000,G,NaN,PG/SG/G,73111-20640:Jordan Clarkson,SG_73111-20640
1,Fred VanVleet,TOR,LAL,37.0,PG,"7,200",42.2,8.4,51.8,30.2,MAIN,Fred VanVleet,PG,73111-24247,7200,G,NaN,PG/G,73111-24247:Fred VanVleet,PG_73111-24247
2,Nikola Jokic,DEN,CLE,34.0,C,"11,200",57.1,7.3,66.7,9.4,MAIN,Nikola Jokic,C,73111-55062,11200,NaN,NaN,C,73111-55062:Nikola Jokic,C_73111-55062
3,CJ McCollum,NOP,SAS,36.0,SG,"8,700",47.1,7.3,54.9,14.6,MAIN,CJ McCollum,SG,73111-19067,8700,G,NaN,SG/G,73111-19067:CJ McCollum,SG_73111-19067


In [22]:
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index_col=['Id', 'FD Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['Salary'].to_dict()
values = df['FD Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 60000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


73111-20640 SG
73111-24247 PG
73111-19067 SG
73111-42669 PF
73111-110357 SF
73111-40203 C
73111-9535 PG
73111-84708 PF
73111-110356 SF


In [24]:
new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]

# Create a lookup variable for [Position + Id]
test['lookup'] = test[1] + "_" + test[0]


# Merge PuLP output on [Position + Id] to pull in Player Name
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv")
upload = pd.merge(test,
                 df[['Player', 'Position + Id', 'Name + ID']],
                 left_on = 'lookup',
                 right_on = 'Position + Id')


# Drop all columns not used in upload
upload2 = upload.drop([0, 'lookup', 'Player', 'Position + Id'], axis = 1)


# Transpose data output from vertical to horizontal
upload3 = upload2.transpose()


# Rename columns to first row (position values)
upload4 = upload3.rename(columns=upload3.iloc[0])


# Drop duplicate first row
upload5 = upload4.drop([1])


# Re-order the columns to the Spositions
upload6 = upload5[['PG',
                   'SG',
                   'SF',
                   'PF',
                   'C']]


# Save as a csv
upload6.to_csv("fd_result_" + str(today) + ".csv", index=None)

# os.system("fd_result_" + str(today) + ".csv")

upload6

C:\Users\vchang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PG,PG,SG,SG,SF,SF,PF,PF,C
Name + ID,73111-24247:Fred VanVleet,73111-9535:Kyle Lowry,73111-20640:Jordan Clarkson,73111-19067:CJ McCollum,73111-110357:RJ Barrett,73111-110356:Nickeil Alexander-Walker,73111-42669:Julius Randle,73111-84708:Robert Williams,73111-40203:Rudy Gobert


In [25]:
upload6 = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\fd_result_' + str(today) + ".csv")
upload6.head()


upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel\FD_Upload_PlayerPool_' + str(today) + ".csv"
                     , error_bad_lines=False, engine ='python')


upload = upload[['entry_id', 'contest_id', 'contest_name']]
upload.head()

os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel')
result = pd.concat([upload, upload6], axis=1)
test = result.rename(columns={"PG.1":"PG", "SG.1":"SG", "SF.1":"SF", "PF.1":"PF"})
result.to_csv("FD_Final_Upload_"+ time + str(today)+".csv", index=False)
os.system("FD_Final_Upload_"+ time + str(today)+".csv")

Skipping line 7: Expected 14 fields in line 7, saw 31
Skipping line 8: Expected 14 fields in line 8, saw 31
Skipping line 9: Expected 14 fields in line 9, saw 31
Skipping line 10: Expected 14 fields in line 10, saw 31
Skipping line 11: Expected 14 fields in line 11, saw 31
Skipping line 12: Expected 14 fields in line 12, saw 31
Skipping line 13: Expected 14 fields in line 13, saw 31
Skipping line 14: Expected 14 fields in line 14, saw 31
Skipping line 15: Expected 14 fields in line 15, saw 31
Skipping line 16: Expected 14 fields in line 16, saw 31
Skipping line 17: Expected 14 fields in line 17, saw 31
Skipping line 18: Expected 14 fields in line 18, saw 31
Skipping line 19: Expected 14 fields in line 19, saw 31
Skipping line 20: Expected 14 fields in line 20, saw 31
Skipping line 21: Expected 14 fields in line 21, saw 31
Skipping line 22: Expected 14 fields in line 22, saw 31
Skipping line 23: Expected 14 fields in line 23, saw 31
Skipping line 24: Expected 14 fields in line 24, saw 3

0

In [26]:
test

,entry_id,contest_id,contest_name,PG,PG,SG,SG,SF,SF,PF,PF,C
0,2.792623e+09,73111-255728507,50-Player League ($1) - Experienced Players Ex...,73111-24247:Fred VanVleet,73111-9535:Kyle Lowry,73111-20640:Jordan Clarkson,73111-19067:CJ McCollum,73111-110357:RJ Barrett,73111-110356:Nickeil Alexander-Walker,73111-42669:Julius Randle,73111-84708:Robert Williams,73111-40203:Rudy Gobert
1,2.792623e+09,73111-255721897,$2.5K Fri NBA EPE Shot (No Experienced Players),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.792623e+09,73111-255734034,NBA Mini Swat (EPE),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
